In [4]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import pandas as pd
import re
from pdfminer.high_level import extract_text
from pdfminer.layout import LAParams
from bs4 import BeautifulSoup
import PyPDF2
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline

def pdf_to_html(pdf_path, html_path):
    text = extract_text(pdf_path, laparams=LAParams())
    html_content = f"<html><body><pre>{text}</pre></body></html>"
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(html_content)

def extract_text_from_pdf(pdf_path):
    extracted_text = ""
    try:  
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    extracted_text += text + "\n"
                else:
                    try: 
                        images = convert_from_path(pdf_path, first_page=page.page_number, last_page=page.page_number)
                        for img in images:
                            extracted_text += pytesseract.image_to_string(img) + "\n"
                    except Exception as e:
                        print(f"Error converting image on page {page.page_number}: {e}")

                        extracted_text += "Error extracting image from this page.\n"
    except Exception as e:
        print(f"Error opening or processing PDF: {e}")
        return None 

    return extracted_text

def parse_financial_data(text):
    revenue = re.findall(r"Revenues\s*[\$]\s*([\d,]+)", text)
    operating_income = re.findall(r"Operating income\s*[\$]\s*([\d,]+)", text)
    net_income = re.findall(r"Net income\s*[\$]\s*([\d,]+)", text)
    
    financial_data = {
        "revenue": revenue,
        "operating_income": operating_income,
        "net_income": net_income
    }
    return financial_data

def summarize_text(text):
    try:
        summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
        max_chunk_size = 1000
        text_chunks = [text[i:i + max_chunk_size] for i in range(0, len(text), max_chunk_size)]
        
        summary = ""
        for chunk in text_chunks:
            chunk_summary = summarizer(chunk, max_length=200, min_length=50, do_sample=False)
            if chunk_summary and isinstance(chunk_summary, list) and "summary_text" in chunk_summary[0]:
                summary += chunk_summary[0]["summary_text"] + " "
            else:
                print("Warning: Summarization did not return any results for a chunk.")
                
        if summary.strip() == "":
            return "No summary available."
        
        return summary.strip()
    except Exception as e:
        print(f"Error during summarization: {e}")
        return "Summarization failed."

def plot_financial_trends(financial_data):
    try:
        if financial_data and financial_data["revenue"] and financial_data["operating_income"]: #check if it exists and has data
            revenue_data = [int(value.replace(",", "")) for value in financial_data["revenue"] if value]
            operating_income_data = [int(value.replace(",", "")) for value in financial_data["operating_income"] if value]
            
            min_length = min(len(revenue_data), len(operating_income_data))
            revenue_data = revenue_data[:min_length]
            operating_income_data = operating_income_data[:min_length]
            
            if revenue_data and operating_income_data: #check if the data lists are not empty
                plt.figure(figsize=(10, 6))
                plt.plot(revenue_data, marker='o', label="Revenue")
                plt.plot(operating_income_data, marker='s', label="Operating Income")
                plt.legend()
                plt.title('Financial Trends (Revenue vs Operating Income)')
                plt.xlabel('Data Points (e.g., Quarters)')
                plt.ylabel('Amount ($)') #Removed millions from y-axis label
                plt.show()
            else:
                print("Insufficient data for plotting financial trends.")
        else:
            print("No revenue or operating income data found to plot.")
    except Exception as e:
        print(f"Error while plotting financial trends: {e}")


if __name__ == "__main__":
    pdf_path = r"C:\Users\Aaarat\Downloads\FINAL-PR_HCA-Reports-4Q-2024-Results.pdf"
    pdf_text = extract_text_from_pdf(pdf_path)

    if pdf_text:
        financial_data = parse_financial_data(pdf_text)
        print("Financial Data Extracted:", financial_data)
        summary = summarize_text(pdf_text)
        print("Document Summary:", summary)
        plot_financial_trends(financial_data)
    else:
        print("No text extracted from PDF; please check the file and path.")

Financial Data Extracted: {'revenue': ['72', '18,285', '70,603', '18,285', '72,800'], 'operating_income': [], 'net_income': ['6,657']}


Device set to use cpu
Your max_length is set to 200, but your input_length is only 190. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=95)
Your max_length is set to 200, but your input_length is only 196. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=98)
Your max_length is set to 200, but your input_length is only 197. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=98)
Your max_length is set to 200, but your input_length is only 194. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer(

Document Summary: HCA announces financial and operating results for the fourth quarter ended December 31, 2024. Revenues totaled $18.285 billion. Net income attributable to HCA Healthcare, Inc. totaled $1.438 billion, or $5.63 per diluted share. Same facility admissions increased 3.0 percent. Revenues in the fourth quarter of 2024 totaled $18.285 billion, compared to $17.303 billion in 2023. Net income attributable to HCA Healthcare, Inc. totaled $1.438 billion, or $5.63 per diluted share. Results include losses on sales of facilities of $195 million. For the fourth quarter of 2024, Adjusted EBITDA totaled $3.712 billion. Same facility admissions increased 3.0 percent and same facility equivalent admissionsincreased 3.1 percent. Same hospital emergency room visits increased 2.4 percent. Revenues for the year ended December 31, 2024 totaled $70.603 billion. Net income attributable to HCA Healthcare, Inc. was $5.760 billion, or $22.00 per diluted share. For 2024, Adjusted EBITDA totaled 

In [8]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import pandas as pd
import re
from pdfminer.high_level import extract_text
from pdfminer.layout import LAParams
from bs4 import BeautifulSoup

def pdf_to_html(pdf_path, html_path):
    text = extract_text(pdf_path, laparams=LAParams())
    html_content = f"<html><body><pre>{text}</pre></body></html>"
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(html_content)

def extract_text_from_pdf(pdf_path):
    extracted_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                extracted_text += text + "\n"
            else:
                images = convert_from_path(pdf_path, first_page=page.page_number, last_page=page.page_number)
                for img in images:
                    extracted_text += pytesseract.image_to_string(img) + "\n"
    return extracted_text

def extract_all_data(text):
    return text

pdf_path = r"C:\Users\Aaarat\Downloads\FINAL-PR_HCA-Reports-4Q-2024-Results.pdf"
html_path = "output.html"
pdf_to_html(pdf_path, html_path)

text = extract_text_from_pdf(pdf_path)
with open("extracted_text.txt", "w", encoding="utf-8") as f:
    f.write(text)

print("Extracted Text from PDF:")
print(text)


Extracted Text from PDF:
FOR IMMEDIATE RELEASE
INVESTOR CONTACT: MEDIA CONTACT:
Frank Morgan Harlow Sumerford
615-344-2688 615-344-1851
HCA HEALTHCARE REPORTS FOURTH QUARTER 2024 RESULTS
AND PROVIDES 2025 GUIDANCE
Nashville, Tenn., January 24, 2025 – HCA Healthcare, Inc. (NYSE: HCA) today announced
financial and operating results for the fourth quarter ended December 31, 2024.
Key fourth quarter metrics (all percentage changes compare 4Q 2024 to 4Q 2023 unless
otherwise noted):
 Revenues totaled $18.285 billion
 Net income attributable to HCA Healthcare, Inc. totaled $1.438 billion, or $5.63 per
diluted share (which includes an estimated $0.60 per diluted share unfavorable impact
from Hurricanes Helene and Milton)
 Adjusted EBITDA totaled $3.712 billion
 Cash flows from operating activities totaled $2.559 billion
 Same facility admissions increased 3.0 percent and same facility equivalent admissions
increased 3.1 percent
"We finished 2024 with strong business fundamentals that wer

In [5]:
text = pdf_text
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    sentences = sent_tokenize(text)
    stop_words = set(stopwords.words('english'))
    processed_sentences = []

    for sentence in sentences:
        words = word_tokenize(sentence)
        filtered_words = [word for word in words if word.lower() not in stop_words]
        processed_sentences.append(' '.join(filtered_words))
    
    return processed_sentences

processed_sentences = preprocess_text(text)
print(processed_sentences)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aaarat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aaarat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['IMMEDIATE RELEASE INVESTOR CONTACT : MEDIA CONTACT : Frank Morgan Harlow Sumerford 615-344-2688 615-344-1851 HCA HEALTHCARE REPORTS FOURTH QUARTER 2024 RESULTS PROVIDES 2025 GUIDANCE Nashville , Tenn. , January 24 , 2025 – HCA Healthcare , Inc. ( NYSE : HCA ) today announced financial operating results fourth quarter ended December 31 , 2024 .', "Key fourth quarter metrics ( percentage changes compare 4Q 2024 4Q 2023 unless otherwise noted ) : \uf0b7 Revenues totaled $ 18.285 billion \uf0b7 Net income attributable HCA Healthcare , Inc. totaled $ 1.438 billion , $ 5.63 per diluted share ( includes estimated $ 0.60 per diluted share unfavorable impact Hurricanes Helene Milton ) \uf0b7 Adjusted EBITDA totaled $ 3.712 billion \uf0b7 Cash flows operating activities totaled $ 2.559 billion \uf0b7 facility admissions increased 3.0 percent facility equivalent admissions increased 3.1 percent '' finished 2024 strong business fundamentals consistent previous quarters , '' said Sam Hazen , Chie

In [6]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

labels = ["Strength", "Weakness", "Opportunity", "Threat"]
swot_results = {"Strengths": [], "Weaknesses": [], "Opportunities": [], "Threats": []}
for sentence in processed_sentences:
    result = classifier(sentence, candidate_labels=labels)
    label = result['labels'][0]
    if label == "Strength":
        swot_results["Strengths"].append(sentence)
    elif label == "Weakness":
        swot_results["Weaknesses"].append(sentence)
    elif label == "Opportunity":
        swot_results["Opportunities"].append(sentence)
    elif label == "Threat":
        swot_results["Threats"].append(sentence)

print(swot_results)


Device set to use cpu


{'Strengths': ["Key fourth quarter metrics ( percentage changes compare 4Q 2024 4Q 2023 unless otherwise noted ) : \uf0b7 Revenues totaled $ 18.285 billion \uf0b7 Net income attributable HCA Healthcare , Inc. totaled $ 1.438 billion , $ 5.63 per diluted share ( includes estimated $ 0.60 per diluted share unfavorable impact Hurricanes Helene Milton ) \uf0b7 Adjusted EBITDA totaled $ 3.712 billion \uf0b7 Cash flows operating activities totaled $ 2.559 billion \uf0b7 facility admissions increased 3.0 percent facility equivalent admissions increased 3.1 percent '' finished 2024 strong business fundamentals consistent previous quarters , '' said Sam Hazen , Chief Executive Officer HCA Healthcare .", '`` first half current decade , ended 2024 , proved another period long-term growth company resulted operational improvements across key performance indicators greater value patients , employees shareholders .', "accomplishments testament incredible work teams , position us well future . ''", 'R

In [7]:
def display_swot_results(swot_results):
    for category, sentences in swot_results.items():
        print(f"\n{category}:\n")
        for sentence in sentences:
            print(f"- {sentence}")

display_swot_results(swot_results)



Strengths:

- Key fourth quarter metrics ( percentage changes compare 4Q 2024 4Q 2023 unless otherwise noted ) :  Revenues totaled $ 18.285 billion  Net income attributable HCA Healthcare , Inc. totaled $ 1.438 billion , $ 5.63 per diluted share ( includes estimated $ 0.60 per diluted share unfavorable impact Hurricanes Helene Milton )  Adjusted EBITDA totaled $ 3.712 billion  Cash flows operating activities totaled $ 2.559 billion  facility admissions increased 3.0 percent facility equivalent admissions increased 3.1 percent '' finished 2024 strong business fundamentals consistent previous quarters , '' said Sam Hazen , Chief Executive Officer HCA Healthcare .
- `` first half current decade , ended 2024 , proved another period long-term growth company resulted operational improvements across key performance indicators greater value patients , employees shareholders .
- accomplishments testament incredible work teams , position us well future . ''
- Revenues fourth quarter 2024 t

In [11]:


def extract_tables_from_pdf(pdf_path):
    tables_data = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                df = pd.DataFrame(table)
                df = df.dropna(how='all').reset_index(drop=True) 
                df.columns = df.iloc[0]
                df = df[1:].reset_index(drop=True)
                tables_data.append(df)
    return tables_data


tables_data = extract_tables_from_pdf(pdf_path)
print("\nExtracted Tables from PDF:")
for i, table in enumerate(tables_data):
    print(f"\nTable {i+1}:")
    print(tabulate(table, headers='keys', tablefmt='fancy_grid'))

for i, table in enumerate(tables_data):
    table.to_csv(f"table_{i+1}.csv", index=False)



Extracted Tables from PDF:

Table 1:
╒════╤════════════╤════╤════╤════╤════════════════════════════╤════╕
│    │ Revenues   │    │    │    │ $72.80 to $75.80 billion   │    │
╞════╪════════════╪════╪════╪════╪════════════════════════════╪════╡
╘════╧════════════╧════╧════╧════╧════════════════════════════╧════╛

Table 2:
╒════╤═══════════════════╤════╤════╤════╤════════════════════════════╤════╕
│    │ Adjusted EBITDA   │    │    │    │ $14.30 to $15.10 billion   │    │
╞════╪═══════════════════╪════╪════╪════╪════════════════════════════╪════╡
╘════╧═══════════════════╧════╧════╧════╧════════════════════════════╧════╛

Table 3:
╒════╤════════════════════════════════════════════════════════════════╤════╤════╤════════╤═════════╤════════╤════════╤════════╤════════╤════╤════╤════╤════════╤═════════╤════════╤════════╤════════╤════════╤════╤════╕
│    │                                                                │    │    │ 2024   │ None    │ None   │ None   │ None   │ None   │    │    

In [14]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error  # Import MSE directly

def create_sequences(data, n_steps):
    """Creates sequences of data for LSTM input."""
    X = []
    y = []
    for i in range(n_steps, len(data)):
        X.append(data[i - n_steps:i])
        y.append(data[i])
    return np.array(X), np.array(y)

def train_lstm_model(X_train, y_train, epochs=50, batch_size=32, verbose=1): #verbose set to 1 by default
    """Trains an LSTM model."""
    n_features = X_train.shape[2]
    timesteps = X_train.shape[1]
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(timesteps, n_features), return_sequences=True))  # First LSTM layer (return_sequences=True for stacking)
    model.add(Dropout(0.2)) # Add dropout for regularization
    model.add(LSTM(50, activation='relu'))  # Second LSTM layer
    model.add(Dropout(0.2)) # Add dropout for regularization
    model.add(Dense(1)) 
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose) #verbose is now a parameter
    return model

def forecast(model, X_test, scaler):
    """Makes forecast using the trained model and inverse transforms."""
    predictions = model.predict(X_test)
    predictions = predictions.reshape(-1, 1)
    predictions = scaler.inverse_transform(np.repeat(predictions, 2, axis=1))
    return predictions

def evaluate_model(y_true, y_pred):
    """Evaluates the model and prints metrics."""
    mse = mean_squared_error(y_true, y_pred) #use sklearn's MSE function
    rmse = np.sqrt(mse)
    print(f"MSE: {mse:.4f}") #format output
    print(f"RMSE: {rmse:.4f}")
    return mse, rmse

def main():
    # Sample Data (replace with your actual data)
    data = {'Date': pd.to_datetime(['2022-01-01', '2022-04-01', '2022-07-01', '2022-10-01', '2023-01-01', '2023-04-01', '2023-07-01']),
            'Current Ratio': [1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1],
            'Quick Ratio': [0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4]}
    df = pd.DataFrame(data)

    features = ['Current Ratio', 'Quick Ratio']
    data = df[features].values

    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)

    n_steps = 3
    X, y = create_sequences(scaled_data, n_steps)

    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    model = train_lstm_model(X_train, y_train)

    predictions = forecast(model, X_test, scaler)
    y_test_original_scale = scaler.inverse_transform(y_test) #inverse transform y_test for evaluation

    mse, rmse = evaluate_model(y_test_original_scale, predictions) #evaluate on original scale

    last_sequence = scaled_data[-n_steps:]
    last_sequence = last_sequence.reshape((1, n_steps, len(features)))
    future_forecast = forecast(model, last_sequence, scaler)
    print("Future Forecast:\n")
    print("Current Ratio:", future_forecast[0][0])
    print("Quick Ratio:",future_forecast[0][1])

if __name__ == "_main_":
    main()

In [15]:
import tkinter as tk
from tkinter import ttk  # For modern widgets
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

def predict_and_display():
    try:
        data = {'Date': pd.to_datetime(['2022-01-01', '2022-04-01', '2022-07-01', '2022-10-01', '2023-01-01', '2023-04-01', '2023-07-01']),
                'Current Ratio': [1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1],
                'Quick Ratio': [0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4]}
        df = pd.DataFrame(data)

        features = ['Current Ratio', 'Quick Ratio']
        data = df[features].values

        # 2. Scale data
        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(data)

        n_steps = 3
        X, y = create_sequences(scaled_data, n_steps)

        train_size = int(len(X) * 0.8)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]

        # 3. Train model
        model = train_lstm_model(X_train, y_train)

        # 4. Make predictions
        predictions = forecast(model, X_test, scaler)
        y_test_original_scale = scaler.inverse_transform(y_test)

        # 5. Evaluate (optional - you can display these too)
        mse, rmse = evaluate_model(y_test_original_scale, predictions)

        # 6. Future forecast
        last_sequence = scaled_data[-n_steps:]
        last_sequence = last_sequence.reshape((1, n_steps, len(features)))
        future_forecast = forecast(model, last_sequence, scaler)

        # 7. Display results in the GUI
        for i, feature in enumerate(features):
            result_label = result_labels[i]  # Get the label for the feature
            forecast_value = future_forecast[0][i]
            result_label.config(text=f"{forecast_value:.4f}")  # Display with formatting

    except Exception as e:
        print(f"Error: {e}")
        error_label.config(text=f"An error occurred: {e}")  # Display error in GUI

root = tk.Tk()
root.title("Financial Forecast")

style = ttk.Style()
style.configure('My.TLabel', font=('Helvetica', 12), justify='center')

# Create labels for features
feature_labels = []
for feature in ['Current Ratio', 'Quick Ratio']:
    label = ttk.Label(root, text=f"{feature}:", style='My.TLabel')  # Apply the style
    label.grid(row=len(feature_labels), column=0, padx=5, pady=5, sticky="ew") # Sticky to expand in east/west
    feature_labels.append(label)

# Create labels to display results
result_labels = []
for _ in ['Current Ratio', 'Quick Ratio']:
    label = ttk.Label(root, text="", style='My.TLabel')  # Apply the style
    label.grid(row=len(result_labels), column=1, padx=5, pady=5, sticky="ew") # Sticky to expand in east/west
    result_labels.append(label)

# Button to trigger prediction
predict_button = ttk.Button(root, text="Predict", command=predict_and_display)
predict_button.grid(row=len(feature_labels), column=0, columnspan=2, pady=10)

error_label = ttk.Label(root, text="", foreground="red", style='My.TLabel')  # Apply style to error label
error_label.grid(row=len(feature_labels) + 1, column=0, columnspan=2, pady=(0, 10))

# Configure grid column weights to center content
root.columnconfigure(0, weight=1)  # Center content in column 0
root.columnconfigure(1, weight=1)


root.mainloop()

C:\Users\Aaarat\PycharmProjects\pythonProject\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 0.4683
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.4572
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.4571
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.4494
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.4459
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.4394
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.4329
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.4259
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.4213
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.4223
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.4078
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.4049
Epoch 13/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.3912
Epoch 14/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.3914
Epoch 15/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.3848
Epoch 16/50
1/1 ━━━━━━━━━━━━━━━━━━━━

C:\Users\Aaarat\PycharmProjects\pythonProject\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.4538
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.4486
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.4419
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.4298
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.4279
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.4243
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.4099
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.4062
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.3936
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.3936
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.3867
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.3825
Epoch 13/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.3809
Epoch 14/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.3706
Epoch 15/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.3435
Epoch 16/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/st